In [3]:
!pip install ccxt
!pip install pandas
!pip install ta
!pip install schedule

  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29422 sha256=d63d7e0cb00117afe666afc24fb2cafeb214a7f9244525dc37196bf69ebf1ead
  Stored in directory: c:\users\eduardo\appdata\local\pip\cache\wheels\5c\a1\5f\c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
  Using cached schedule-1.2.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached schedule-

In [3]:
#import packages

import ccxt

import pandas as pd
pd.set_option('display.max_rows', None)

from datetime import datetime

from ta.trend import MACD
from ta.momentum import RSIIndicator
import warnings
warnings.filterwarnings('ignore')

import schedule as schedule
import time


In [4]:
#data retrival and order execution
exchange = ccxt.binance()

In [11]:
def execute_connection(symbol='ETH/USDT', timeframe='1m'):
    '''
        function for data retrival, processing and cleaning
    '''
    raw_data = exchange.fetch_ohlcv(symbol, timeframe, limit=100)
    
    # print(raw_data)
    
    # skip todays date
    df = pd.DataFrame(raw_data[:-1], columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    print(f"Executing connection and data processing at... {datetime.now().isoformat()}")
    print(df)

In [13]:
execute_connection(symbol='ETH/USDT', timeframe='15m')

Executing connection and data processing at... 2024-03-20T21:27:56.789406
                  date     open     high      low    close      volume
0  2024-03-20 02:30:00  3219.00  3223.90  3206.00  3221.65   7437.3096
1  2024-03-20 02:45:00  3221.62  3228.73  3211.87  3224.32   6957.0456
2  2024-03-20 03:00:00  3224.33  3235.00  3222.00  3224.57   8815.3798
3  2024-03-20 03:15:00  3224.57  3266.01  3220.60  3263.99  15919.1709
4  2024-03-20 03:30:00  3263.98  3264.25  3231.32  3232.74   6811.9087
5  2024-03-20 03:45:00  3232.73  3236.66  3153.26  3161.02  22745.6704
6  2024-03-20 04:00:00  3161.02  3194.24  3154.40  3191.38  15374.0355
7  2024-03-20 04:15:00  3191.38  3191.68  3157.05  3159.85  11694.2564
8  2024-03-20 04:30:00  3159.84  3161.01  3099.99  3120.23  36464.9662
9  2024-03-20 04:45:00  3120.22  3137.38  3116.70  3129.11  10738.6037
10 2024-03-20 05:00:00  3129.10  3140.65  3089.43  3101.70  15933.0011
11 2024-03-20 05:15:00  3101.70  3108.08  3056.56  3074.22  18683.6908
12 

In [15]:
def technical_signals(df):
    '''
    this function will implement the technical indicator to the market DataFrame.
    the second par will define the logics with a bolean final.
    '''
    
    # Manual MACD
    # ShortEMA = df['close'].ewm(span=12, adjust=False).mean()
    # LongEMA = df['close'].ewm(span=26, adjust=False).mean()
    # MACD = ShortEMA - LongEMA
    # Signal = MACD.ewm(span=9, adjust=False).mean()
    # df['MACD'] = MACD
    # df['Signal'] = Signal
    
    # Get Indicators using libary MACD & RSI for the example
    # MACD
    indicator_macd = MACD(df['close'])
    df['MACD'] = indicator_macd.macd()
    df['Signal'] = indicator_macd.macd_signal()
    df['MACD Histogram'] = indicator_macd.macd_diff()
    
    df['MACD_Signal'] = False
    
    # RSI
    indicator_rsi = RSIIndicator(df['close'], window=14)
    df['RSI_Signal'] = False
    df['RSI'] = indicator_rsi.rsi()
    
    
    # Technical indicator strategy logics. We will use simple MACD to detect signals:
    # Crossover MACD over signal Below 0 == Buy Signal
    # Crossover MACD under Signal Above 0 == Sell Signal
    for current in range(1, len(df.index)):
        previous = current - 1
        if (df['MACD'][current] > df['Signal'][current] and df['MACD'][previous] < df['Signal'][previous] and df['MACD'][current] < 0):
            df['MACD_Signal'][current] = True
        elif (df['MACD'][current] < df['Signal'][current] and df['MACD'][previous] > df['Signal'][previous]):
            df['MACD_Signal'][current] = False
        else:
            df['MACD_Signal'][current] = df['MACD_Signal'][previous]
    return df
        

In [16]:
#  Defines if we're in hte market or not, to avoid submit orders once wwe've submited one
in_position = False
def reading_market(df):
    '''
    function to analize the market looking for signals according to our logics
    '''
    global in_position
    
    print("Looking for signals...")
    print(df.tail(5))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1
    
    if not df['MACD_Signal'][previous_row_index] and df['MACD_Signal'][last_row_index]:
        print('Uptrend activated according MACD, BUY SIGNAL triggered')
        if not in_position:
            order_buy = "Here goes BUY order" # exchange.create_market_buy_order('ETH/USDT', 1)
            print(order_buy)
            in_position = True
        else: 
            print("Already in position, skip BUY signal")
    
    if df['MACD_Signal'][previous_row_index] and not df['MACD_Signal'][last_row_index]:
        print('Downtrend activated according MACD, SELL SIGNAL triggered')
        if not in_position:
            order_sell = "Here goes SELL order" # exchange.create_market_sell_order('ETH/USDT', 1)
            print(order_sell)
            in_position = False
        else: 
            print("Not in position, skip SELL signal")

In [17]:
def execute_connection(symbol='ETH/USDT', timeframe='1m'):
    '''
        function for data retrival, processing and cleaning
    '''
    raw_data = exchange.fetch_ohlcv(symbol, timeframe, limit=100)
    
    # print(raw_data)
    
    # skip todays date
    df = pd.DataFrame(raw_data[:-1], columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    print(f"Executing connection and data processing at... {datetime.now().isoformat()}")
    # print(df)
    complete_df = technical_signals(df)
    reading_market(complete_df)

In [18]:
schedule.every(10).seconds.do(execute_connection)

while True:
    schedule.run_pending()
    time.sleep(1)

Executing connection and data processing at... 2024-03-20T22:03:14.081663
                  date     open     high      low    close     volume
0  2024-03-21 02:24:00  3544.62  3547.17  3542.05  3547.17   313.6311
1  2024-03-21 02:25:00  3547.16  3547.17  3543.65  3543.66   332.0833
2  2024-03-21 02:26:00  3543.66  3545.54  3543.21  3545.53   144.5398
3  2024-03-21 02:27:00  3545.53  3545.54  3542.00  3544.77   239.9873
4  2024-03-21 02:28:00  3544.77  3545.54  3542.87  3544.59   226.7025
5  2024-03-21 02:29:00  3544.59  3546.98  3543.60  3546.15   227.0244
6  2024-03-21 02:30:00  3546.14  3549.05  3545.63  3545.64   189.6046
7  2024-03-21 02:31:00  3545.64  3546.63  3543.29  3543.29   120.4913
8  2024-03-21 02:32:00  3543.30  3546.88  3542.61  3546.87   167.8709
9  2024-03-21 02:33:00  3546.88  3555.54  3546.87  3555.48   438.0656
10 2024-03-21 02:34:00  3555.48  3556.85  3552.60  3553.06   515.9232
11 2024-03-21 02:35:00  3553.07  3555.00  3551.84  3554.98   191.7308
12 2024-03-21 02

KeyboardInterrupt: 